### Welcome to the program
Anything you will need will be in this file.

First let some modules to download

In [9]:
!pip install ipywidgets;


In [10]:
import ipywidgets as widgets


from fairness_metrics.predicted_probs.balance_in_pos_neg import balance_in_pos_neg

In [11]:
a = balance_in_pos_neg(True, False)
a.print()

AttributeError: 'balance_in_pos_neg' object has no attribute 'print'